1. Import data into dataframe
2. Label annotated observations properly
3. Build classifier

First we import the data into a dataframe

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import pyinaturalist
import time
import requests

In [5]:
flowerObs = pd.read_csv("rawData/occurrence.txt",sep='\t')

C:\Users\Sean Haight\AppData\Local\Temp\ipykernel_11260\4254718079.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  flowerObs = pd.read_csv("rawData/occurrence.txt",sep='\t')


In [7]:
list(flowerObs.columns)

['gbifID',
 'accessRights',
 'bibliographicCitation',
 'language',
 'license',
 'modified',
 'publisher',
 'references',
 'rightsHolder',
 'type',
 'institutionID',
 'collectionID',
 'datasetID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'ownerInstitutionCode',
 'basisOfRecord',
 'informationWithheld',
 'dataGeneralizations',
 'dynamicProperties',
 'occurrenceID',
 'catalogNumber',
 'recordNumber',
 'recordedBy',
 'recordedByID',
 'individualCount',
 'organismQuantity',
 'organismQuantityType',
 'sex',
 'lifeStage',
 'reproductiveCondition',
 'behavior',
 'establishmentMeans',
 'degreeOfEstablishment',
 'pathway',
 'georeferenceVerificationStatus',
 'occurrenceStatus',
 'preparations',
 'disposition',
 'associatedOccurrences',
 'associatedReferences',
 'associatedSequences',
 'associatedTaxa',
 'otherCatalogNumbers',
 'occurrenceRemarks',
 'organismID',
 'organismName',
 'organismScope',
 'associatedOrganisms',
 'previousIdentifications',
 'organismRemarks',
 'materialSam

Now we drop the extra columns

In [58]:
flowerObs = flowerObs.drop(columns=['continent', 'stateProvince', 'countryCode', 'coordinateUncertaintyInMeters', 'identificationID', 'identifiedBy', 'identifiedByID', 'dateIdentified'])
flowerObs

,gbifID,references,reproductiveCondition,year,month,day,decimalLatitude,decimalLongitude,species
0,923926423,https://www.inaturalist.org/observations/765206,NaN,2014,6,26,42.045078,-79.842641,Rosa multiflora
1,923922892,https://www.inaturalist.org/observations/754077,NaN,2014,6,25,42.059677,-87.841293,Rosa blanda
2,923922820,https://www.inaturalist.org/observations/754075,NaN,2014,6,25,42.059722,-87.842464,Rosa carolina
3,923919411,https://www.inaturalist.org/observations/745120,NaN,2014,6,19,41.162646,-81.561961,Rosa multiflora
4,923916102,https://www.inaturalist.org/observations/737244,NaN,2014,6,14,38.610080,-122.771294,Rosa californica
...,...,...,...,...,...,...,...,...,...
58541,1024202752,https://www.inaturalist.org/observations/814177,NaN,2014,7,31,44.463158,-110.826821,Rosa woodsii
58542,1024201977,https://www.inaturalist.org/observations/812446,flowering,2014,7,31,41.542486,-70.601946,Rosa rugosa
58543,1024191118,https://www.inaturalist.org/observations/786474,NaN,2014,7,15,39.433804,-120.203799,Rosa woodsii
58544,1024182125,https://www.inaturalist.org/observations/725899,NaN,2014,6,1,40.655151,-73.993449,Rosa multiflora


Now we write to file

In [59]:
flowerObs.to_csv("processedData/roseObs")

Now we can start creating our model

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
import pyinaturalist as iNat
roseObs = pd.read_csv("processedData/roseObs")
labeledRoseObs = roseObs.dropna()
labeledRoseObs["isFlowering"] = labeledRoseObs['reproductiveCondition'].isin(['flowering','flowering|fruiting','flowering|fruiting|flower budding','flowering|flower budding']).astype(int)
numList = []

for link in labeledRoseObs["references"]:
    numList.append(int(link.split("/")[-1]))

labeledRoseObs["observationNumber"] = numList

C:\Users\Sean Haight\AppData\Local\Temp\ipykernel_17820\248044967.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeledRoseObs["isFlowering"] = labeledRoseObs['reproductiveCondition'].isin(['flowering','flowering|fruiting','flowering|fruiting|flower budding','flowering|flower budding']).astype(int)
C:\Users\Sean Haight\AppData\Local\Temp\ipykernel_17820\248044967.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeledRoseObs["observationNumber"] = numList


Maybe try to use log10 in your analysis. We can also compute the correlation. We could compare mean and median of variables as a function of categorical variables.Even better we can make a box and whisker plot.

In [30]:
kfold = KFold(n_splits=5, random_state=44, shuffle=True)

accuracy = np.zeros((4, 5))
## sets a split counter
i = 0

## loop through the kfold here
for train_index, test_index in kfold.split(labeledRoseObs):
    ## get the cv training set
    roses_tt = labeledRoseObs.iloc[train_index]
    
    ## get the cv holdout set
    roses_ho = labeledRoseObs.iloc[test_index]
    
    ## "Fit" and get ho mse for the baseline model
    baseline_pred = []
    for month in roses_ho["month"]:
        # a better way to do this would be to check which months have a majority of observations in flower
        if month in [5,6,7]:
            baseline_pred.append(1)
        else:
            baseline_pred.append(0)
    
    accuracy[0, i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(baseline_pred)))/len(roses_ho)
    
    ## Fit and get the mses for the remaining models
    LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = LR.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[1,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)
    
    RF = RandomForestClassifier(n_estimators= 200, max_depth=400, random_state=0)
    RF.fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = RF.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[2,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)

    GB = GradientBoostingClassifier(n_estimators = 400, max_leaf_nodes = 10, max_depth =  None, random_state = 2, min_samples_split = 10)
    GB.fit(roses_tt[["year", "month", "day", "decimalLatitude", "decimalLongitude"]], roses_tt["isFlowering"])
    pred = GB.predict(roses_ho[["year", "month", "day", "decimalLatitude", "decimalLongitude"]])
    accuracy[3,i] = 1 - sum(abs(np.array(roses_ho["isFlowering"]) - np.array(pred)))/len(roses_ho)

    
    ## Increasing the counter
    i = i + 1

accuracy


array([[0.80792952, 0.7876652 , 0.79207048, 0.78854626, 0.79118943],
       [0.78942731, 0.77092511, 0.77180617, 0.75242291, 0.77709251],
       [0.86960352, 0.86960352, 0.85814978, 0.86167401, 0.86343612],
       [0.8784141 , 0.87136564, 0.85462555, 0.85638767, 0.87400881]])

In [31]:
np.mean(accuracy,axis = 1)

array([0.79348018, 0.7723348 , 0.86449339, 0.86696035])

For now lets use baseline_pred as our control. The next step is to try to obtain the photos for each of our labeled observations. 

In [7]:
labeledRoseObs

,Unnamed: 0,gbifID,references,reproductiveCondition,year,month,day,decimalLatitude,decimalLongitude,species,isFlowering
6,6,923911800,https://www.inaturalist.org/observations/528175,flowering,2011,7,12,43.674670,-73.235657,Rosa palustris,1
33,33,891740644,https://www.inaturalist.org/observations/397958,fruiting,2013,9,14,37.840454,-122.186905,Rosa gymnocarpa,0
64,64,4407667309,https://www.inaturalist.org/observations/18188...,fruiting,2023,9,2,44.270201,-72.144639,Rosa sherardii,0
65,65,4407660609,https://www.inaturalist.org/observations/18180...,fruiting,2023,9,4,43.192559,-77.513277,Rosa multiflora,0
94,94,4407561413,https://www.inaturalist.org/observations/18117...,flowering|fruiting,2023,9,1,60.435381,22.263368,Rosa rugosa,1
...,...,...,...,...,...,...,...,...,...,...,...
58482,58482,1088892956,https://www.inaturalist.org/observations/751139,flowering,2014,6,7,41.891103,-89.341404,Rosa carolina,1
58489,58489,1052606279,https://www.inaturalist.org/observations/1164586,flowering,2014,12,29,-44.362680,169.802258,Rosa rubiginosa,1
58491,58491,1052593426,https://www.inaturalist.org/observations/1132071,flowering,2014,12,11,-41.082042,175.354832,Rosa rubiginosa,1
58542,58542,1024201977,https://www.inaturalist.org/observations/812446,flowering,2014,7,31,41.542486,-70.601946,Rosa rugosa,1


In [4]:
labeledRoseObs

,Unnamed: 0,gbifID,references,reproductiveCondition,year,month,day,decimalLatitude,decimalLongitude,species,isFlowering,observationNumber
6,6,923911800,https://www.inaturalist.org/observations/528175,flowering,2011,7,12,43.674670,-73.235657,Rosa palustris,1,528175
33,33,891740644,https://www.inaturalist.org/observations/397958,fruiting,2013,9,14,37.840454,-122.186905,Rosa gymnocarpa,0,397958
64,64,4407667309,https://www.inaturalist.org/observations/18188...,fruiting,2023,9,2,44.270201,-72.144639,Rosa sherardii,0,181884043
65,65,4407660609,https://www.inaturalist.org/observations/18180...,fruiting,2023,9,4,43.192559,-77.513277,Rosa multiflora,0,181805827
94,94,4407561413,https://www.inaturalist.org/observations/18117...,flowering|fruiting,2023,9,1,60.435381,22.263368,Rosa rugosa,1,181178504
...,...,...,...,...,...,...,...,...,...,...,...,...
58482,58482,1088892956,https://www.inaturalist.org/observations/751139,flowering,2014,6,7,41.891103,-89.341404,Rosa carolina,1,751139
58489,58489,1052606279,https://www.inaturalist.org/observations/1164586,flowering,2014,12,29,-44.362680,169.802258,Rosa rubiginosa,1,1164586
58491,58491,1052593426,https://www.inaturalist.org/observations/1132071,flowering,2014,12,11,-41.082042,175.354832,Rosa rubiginosa,1,1132071
58542,58542,1024201977,https://www.inaturalist.org/observations/812446,flowering,2014,7,31,41.542486,-70.601946,Rosa rugosa,1,812446


In [5]:
iNat.Observation.from_json_list(iNat.get_observation(1132071))[0].photos[0].thumbnail_url

'https://inaturalist-open-data.s3.amazonaws.com/photos/1423313/square.JPG'

In [61]:
url = 'https://inaturalist-open-data.s3.amazonaws.com/photos/497960/square.jpg'
len(photoURLList[:50])

50

In [3]:
i = 0
for obNum in labeledRoseObs["observationNumber"][1500:2000]:

    response = requests.get(iNat.Observation.from_json_list(iNat.get_observation(obNum))[0].photos[0].thumbnail_url)

    with open(str("photos" + "/" + str(obNum)) + ".jpg", "wb") as f:
        f.write(response.content)

    print(i)
    i = i + 1
    time.sleep(3)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27